In [1]:
import sparknlp

from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetector, BertSentenceEmbeddings
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

In [2]:
spark = sparknlp.start()

24/04/17 18:19:49 WARN Utils: Your hostname, alber-victus resolves to a loopback address: 127.0.1.1; using 10.251.208.55 instead (on interface wlp4s0)
24/04/17 18:19:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/alber/.ivy2/cache
The jars for the packages stored in: /home/alber/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b9c4b298-0e51-4e2f-b0d0-9887eeb301df;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central


In [3]:
df = spark.read.option("header","true").option("inferschema","true").json("../../data/json/Tesla/")

In [7]:
documentAssembler = DocumentAssembler() \
    .setInputCol("article") \
    .setOutputCol("document")

sentence = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("article")

embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_128") \
    .setInputCols(["article"]) \
    .setOutputCol("article_bert_embeddings")\
    .setCaseSensitive(True) \
    .setMaxSentenceLength(512)

pipeline = Pipeline(stages=[documentAssembler,
                            sentence,
                            embeddings])

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[ | ]sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
Download done! Loading the resource.
[ / ]

2024-04-17 18:23:10.868466: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


In [8]:
text_df = df.select(['Published Date','article'])

model = pipeline.fit(text_df)
result_bert = model.transform(text_df)

In [9]:
result_df = result_bert.select(F.explode(F.arrays_zip
                        (result_bert.article.result, 
                         result_bert.article_bert_embeddings.embeddings)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("article"),
                          F.expr("cols['1']").alias("Bert_article_embeddings"))
# result_df.show(truncate=150)

In [10]:
result_df.columns

['article', 'Bert_article_embeddings']

In [11]:
result_df.show(1)

+--------------------+-----------------------+
|             article|Bert_article_embeddings|
+--------------------+-----------------------+
|Written by: Dr. S...|   [-1.3972816, 0.49...|
+--------------------+-----------------------+
only showing top 1 row



In [13]:
# array = result_df.select("Bert_sentence_embeddings").take(2)[1].Bert_sentence_embeddings#.collect()

In [14]:
sentiment_df = text_df.join(result_df,on='article')

In [15]:
sentiment_df.take(1)

[Row(article="Nicholas Hoult unveils giant footprint\n\nActor Nicholas Hoult is in talks to play inventor Nikolai Tesla in the Weinstein Company's 'The Current War'.Benedict Cumberbatch is already on board to play Thomas Edison and Michael Shannon will portray George Westinghouse in the film, to be directed by Alfonso Gomez-Rejon, reported Entertainment Weekly.The movie is set in late 1880s and revolves around power titans Edison and Westinghouse's battle over the supply of electricity.Edison mastered the use of a direct current for electric power distribution over an alternating current, which was backed by several European companies and Westinghouse Electric.Timur Bekmambetov, who was originally eyed to direct the movie, will produce alongside Film Rites' Steve Zaillian and Garrett Basch.", Published Date='Sun, 29 Jan 2017 08:00:00 GMT', Bert_article_embeddings=[-0.22308900952339172, 0.5655825734138489, 0.2038702368736267, -0.560979425907135, 0.13929374516010284, -0.4191528260707855,